# Motif and Transcription Factor enrichment after Mowgli integration

This notebook sums up a few guidelines to 

* extract the top features (e.g., genes or peaks) for all dimensions in the Mowgli embedding.
* perform a motif enrichment analysis (using peaks) (as done for Figure 5 of our [manuscript](https://www.nature.com/articles/s41467-023-43019-2)).

The  code was used in the [original publication](https://doi.org/10.1038/s41467-023-43019-2) to perform motif enrichment analysis from chromatin accessibility (Figure 5-C).  This notebook is a summarisation of the code that is stored in our [Mowgli reproducibility](https://github.com/cantinilab/mowgli_reproducibility) repository.

**NOTE:** This notebook uses both R and Python code. We recommend to copy paste in your local Jupyter or Rstudio session and run the code in the corresponding language. 
**NOTE #2:** The enrichment are performed on human data. Change this code and the databases accordingly if you are working with other species. 

## Extract top features of a modality from Mowgli

We assume that the results of Mowgli integration are in a `mdata` object that store genes in the `rna` and peaks in the `atac` slot.

In [ ]:
# Python
# method to extract the top features
def top_mowgli(dim, n, H_mowgli):
    """
    Get the top n peaks for a given dimension.
    """
    H_scaled = H_mowgli / H_mowgli.sum(axis=1, keepdims=True)
    return H_scaled[:, dim].argsort()[::-1][:n]

### Extract the top peaks

In [ ]:
# Python
# Peaks

# we set the number of peaks to look at
n_peaks = 100

# Get the genes or peak dictionaries
H_mowgli_atac = mdata["atac"].uns["H_OT"]

# actual features extraction
mdata["atac"].var_names = mdata["atac"].var_names.str.replace("atac:", "")
top_in_mowgli = mdata["atac"].var.copy()

# Fill the Mowgli top peaks.
for dim in range(H_mowgli_atac.shape[1]):
    col_name = f"top_in_dim_{dim}"
    idx = top_in_mowgli.index[top_mowgli(dim, n_peaks, H_mowgli_atac)]
    top_in_mowgli[col_name] = False
    top_in_mowgli.loc[idx, col_name] = True

# Save Mowgli's top peaks.
top_in_mowgli.to_csv("top_peaks_in_mowgli.csv")

### Extract the top genes (for other expression-space based enrchments)

In [ ]:
# Python
# we set the number of peaks to look at
n_genes = 100

H_mowgli_rna = mdata["rna"].uns["H_OT"]

# select the top genes to probe using only the highly variable genes (our universe)
top_in_mowgli = (
    mdata["rna"].var.loc[mdata["rna"].var["highly_variable"] == True, :].copy()
)  # the var coordinates

for dim in range(H_mowgli_rna.shape[1]):
    print(dim)
    # name of the column iun the var object that will be used to extract the top peaks for each gfiven dimenssion
    col_name = f"top_in_dim_{dim}"
    idx = top_in_mowgli.index[
        top_mowgli(dim, n_genes, H_mowgli_rna)
    ]  # indices of the top features for that given dimensions. will be used for localizing the peaks afterwasrds
    top_in_mowgli[col_name] = False  # set all value for that dimesions to False
    top_in_mowgli.loc[idx, col_name] = True  # set to True only the peaks that are

# Save Mowgli's top genes.
top_in_mowgli.to_csv(
    os.path.join(top_feats_dir, f"top_genes_in mowgli.csv"),
)

## Motif enrichment 

In [ ]:
# R
# Imports.
library(GenomicRanges)
library(motifmatchr)
library(chromVAR)
library(TFBSTools)
library(JASPAR2022)
library(Signac)
library(BSgenome.Hsapiens.UCSC.hg38)
library(chromVARmotifs)
library(MuData)

In [ ]:
# R
# Read atac file.
in_atac <- "top_peaks_in_mowgli.csv" # nolint
peaks_csv <- read.csv(in_atac, row.names = 2)

In [ ]:
# R
# Optional: Remove non-canonical chromosomes.
peaks_csv < -peaks_csv[peaks_csv["Chromosome"] != "GL000194.1",]
peaks_csv < -peaks_csv[peaks_csv["Chromosome"] != "GL000205.2",]
peaks_csv < -peaks_csv[peaks_csv["Chromosome"] != "GL000205.2",]
peaks_csv < -peaks_csv[peaks_csv["Chromosome"] != "GL000219.1",]
peaks_csv < -peaks_csv[peaks_csv["Chromosome"] != "GL000219.1",]
peaks_csv < -peaks_csv[peaks_csv["Chromosome"] != "KI270721.1",]
peaks_csv < -peaks_csv[peaks_csv["Chromosome"] != "KI270726.1",]
peaks_csv < -peaks_csv[peaks_csv["Chromosome"] != "KI270726.1",]
peaks_csv < -peaks_csv[peaks_csv["Chromosome"] != "KI270713.1",]

In [ ]:
# R
# Convert the peaks to GRanges.
chromosomes <- peaks_csv["Chromosome"][, 1]
ranges <- IRanges::IRanges(
    start = peaks_csv["Start"][, 1],
    end = peaks_csv["End"][, 1]
)
peaks <- GenomicRanges::GRanges(seqnames = chromosomes, ranges = ranges)

In [ ]:
# R
# Get JASPAR motifs.
opts <- list()
opts["species"] <- "Homo sapiens"
opts["collection"] <- "CORE"
motifs <- TFBSTools::getMatrixSet(JASPAR2022::JASPAR2022, opts)
motifs_pwm <- TFBSTools::toPWM(motifs)

# Get cisBP motifs.
data("human_pwms_v2")

# Fuse JASPAR and cisBP motifs.
for (name in names(motifs_pwm)) {
    human_pwms_v2[name] <- motifs_pwm[name]
}

In [ ]:
# R
# Create a 'dummy' Signac object from the peaks.
# Actually giving peaks_csv is nonsense.
# But we only care about the rownames so it's fine.
assay <- Signac::CreateChromatinAssay(
    peaks_csv,
    ranges = peaks,
    sep = c(":", "-")
)

# Create statistics about peaks.
assay <- Signac::RegionStats(
    object = assay,
    genome = BSgenome.Hsapiens.UCSC.hg38
)

# Add the downloaded motif PWM annotation.
assay <- Signac::AddMotifs(
    object = assay,
    genome = BSgenome.Hsapiens.UCSC.hg38,
    pfm = human_pwms_v2
)

In [ ]:
# R
# Define where to save the motif enrichment outputs.
out_motif <- "motifs_"

# Get all top peaks.
background <- c()
for (dim in 0:49) {

    # Get the top peaks for that dimension.
    features <- rownames(assay)[peaks_csv[paste0("top_in_dim_", dim)] == "True"]

    background <- c(background, features)
}

# Iterate over Mowgli's dimensions.
for (dim in 0:49) {

    # Get the top peaks for that dimension.
    features <- rownames(assay)[peaks_csv[paste0("top_in_dim_", dim)] == "True"]

    # Do motif enrichment analysis.
    enriched_motifs <- Signac::FindMotifs(
        object = assay,
        features = features,
        background = background
    )

    # Save the enrichment.
    write.csv(enriched_motifs, paste0(out_motif, dim, ".csv"))
}